In [1]:
# Importing necessary libraries
import cv2
import numpy as np
import pandas as pd
import os
import datetime
from tqdm import tqdm
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# Load Dataset

In [2]:
import os
from tensorflow.keras.preprocessing.image import load_img, img_to_array

# Dataset directory
dataset_path = r'D:\Gender Age Detector\utkface_gender_split'
IMG_SIZE = 128

images = []
ages = []
genders = []

# Go through each gender folder
for gender_label, gender_name in enumerate(['male', 'female']):
    gender_folder = os.path.join(dataset_path, gender_name)
    for filename in os.listdir(gender_folder):
        if filename.endswith('.jpg') or filename.endswith('.png'):
            try:
                # Extract age from filename (format: age_gender_otherinfo.jpg)
                age = int(filename.split('_')[0])

                # Load and preprocess image
                img_path = os.path.join(gender_folder, filename)
                img = load_img(img_path, target_size=(IMG_SIZE, IMG_SIZE))
                img_array = img_to_array(img) / 255.0  # Normalize

                images.append(img_array)
                ages.append(age)
                genders.append(gender_label)  # 0 for male, 1 for female
            except Exception as e:
                print(f"Skipping file {filename} due to error: {e}")

# Preprocess

In [3]:
images = np.array(images, dtype="float32")
ages = np.array(ages, dtype="int32")
genders = np.array(genders, dtype="int32")
genders = to_categorical(genders, num_classes=2)

# Split 

In [4]:
from sklearn.model_selection import train_test_split

# Split for both age and gender
X_train, X_test, age_train, age_test, gender_train, gender_test = train_test_split(
    images, ages, genders, test_size=0.2, random_state=42
)

In [5]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

IMG_SIZE = 128
BATCH_SIZE = 32

datagen = ImageDataGenerator(validation_split=0.2, rescale=1./255)

train_gen = datagen.flow_from_directory(
    directory=dataset_path,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training'
)

val_gen = datagen.flow_from_directory(
    directory=dataset_path,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation'
)

Found 18967 images belonging to 2 classes.
Found 4741 images belonging to 2 classes.


In [6]:
from tensorflow.keras.utils import Sequence
import cv2
import numpy as np
import os

class AgeGenderDataGenerator(Sequence):
    def __init__(self, folder_path, batch_size=32, img_size=128, shuffle=True):
        self.folder_path = folder_path
        self.batch_size = batch_size
        self.img_size = img_size
        self.shuffle = shuffle
        self.image_paths = []
        self.ages = []
        self.genders = []

        # Load image paths and extract labels
        for gender_label, gender_name in enumerate(['male', 'female']):
            gender_folder = os.path.join(folder_path, gender_name)
            for filename in os.listdir(gender_folder):
                if filename.endswith('.jpg') or filename.endswith('.png'):
                    try:
                        age = int(filename.split('_')[0])
                        img_path = os.path.join(gender_folder, filename)

                        self.image_paths.append(img_path)
                        self.ages.append(age)
                        self.genders.append(gender_label)
                    except:
                        continue

        self.indexes = np.arange(len(self.image_paths))
        if self.shuffle:
            np.random.shuffle(self.indexes)

    def __len__(self):
        return int(np.ceil(len(self.image_paths) / self.batch_size))

    def __getitem__(self, idx):
        batch_indexes = self.indexes[idx * self.batch_size:(idx + 1) * self.batch_size]

        batch_images = []
        batch_ages = []
        batch_genders = []

        for i in batch_indexes:
            img = cv2.imread(self.image_paths[i])
            img = cv2.resize(img, (self.img_size, self.img_size))
            img = img.astype('float32') / 255.0

            batch_images.append(img)
            batch_ages.append(self.ages[i])
            batch_genders.append(self.genders[i])

        batch_images = np.array(batch_images)
        batch_ages = np.array(batch_ages)
        batch_genders = np.array(batch_genders)

        # One-hot encode gender
        batch_genders = np.eye(2)[batch_genders]

        return batch_images, {'age_output': batch_ages, 'gender_output': batch_genders}

    def on_epoch_end(self):
        if self.shuffle:
            np.random.shuffle(self.indexes)

In [7]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam

# Input layer
input_layer = Input(shape=(128, 128, 3))

# Shared CNN base
x = Conv2D(32, (3, 3), activation='relu')(input_layer)
x = MaxPooling2D((2, 2))(x)
x = Conv2D(64, (3, 3), activation='relu')(x)
x = MaxPooling2D((2, 2))(x)
x = Conv2D(128, (3, 3), activation='relu')(x)
x = MaxPooling2D((2, 2))(x)
x = Flatten()(x)
x = Dropout(0.5)(x)

# Age prediction branch (regression)
age_output = Dense(1, name='age_output')(x)

# Gender prediction branch (classification)
gender_output = Dense(2, activation='softmax', name='gender_output')(x)

# Define model
model = Model(inputs=input_layer, outputs=[age_output, gender_output])

# Compile model
model.compile(optimizer=Adam(learning_rate=0.001),
              loss={'age_output': 'mse', 'gender_output': 'categorical_crossentropy'},
              metrics={'age_output': 'mae', 'gender_output': 'accuracy'})

In [8]:
# Create training and validation generators
dataset_path = r'D:\Gender Age Detector\utkface_gender_split'
train_gen = AgeGenderDataGenerator(dataset_path, batch_size=32, img_size=128, shuffle=True)
val_gen = AgeGenderDataGenerator(dataset_path, batch_size=32, img_size=128, shuffle=True)

# Train the model
from tensorflow.keras.callbacks import EarlyStopping

early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

history = model.fit(train_gen,
                    validation_data=val_gen,
                    epochs=10,
                    callbacks=[early_stop])

Epoch 1/10
741/741 [==============================] - 479s 645ms/step - loss: 349.7587 - age_output_loss: 348.9314 - gender_output_loss: 0.8273 - age_output_mae: 14.2127 - gender_output_accuracy: 0.6709 - val_loss: 243.4281 - val_age_output_loss: 242.9410 - val_gender_output_loss: 0.4873 - val_age_output_mae: 11.4145 - val_gender_output_accuracy: 0.8246
Epoch 2/10
741/741 [==============================] - 464s 626ms/step - loss: 216.4304 - age_output_loss: 215.7321 - gender_output_loss: 0.6982 - age_output_mae: 11.2889 - gender_output_accuracy: 0.7699 - val_loss: 182.9880 - val_age_output_loss: 182.6023 - val_gender_output_loss: 0.3856 - val_age_output_mae: 10.0984 - val_gender_output_accuracy: 0.8524
Epoch 3/10
741/741 [==============================] - 480s 648ms/step - loss: 178.6194 - age_output_loss: 177.9610 - gender_output_loss: 0.6583 - age_output_mae: 10.2479 - gender_output_accuracy: 0.7926 - val_loss: 183.4068 - val_age_output_loss: 183.0197 - val_gender_output_loss: 0.3870

In [9]:
# Evaluate the model on the validation generator
results = model.evaluate(val_gen)

# Print each metric name and its value
for name, value in zip(model.metrics_names, results):
    print(f"{name}: {value}")

741/741 [==============================] - 79s 106ms/step - loss: 87.1893 - age_output_loss: 86.9618 - gender_output_loss: 0.2275 - age_output_mae: 6.9763 - gender_output_accuracy: 0.9112
loss: 87.18933868408203
age_output_loss: 86.96175384521484
gender_output_loss: 0.227542445063591
age_output_mae: 6.97626256942749
gender_output_accuracy: 0.9112114310264587


In [10]:
# Save the trained model
model.save('age_gender_model.h5')

C:\Users\VISHWA\AppData\Roaming\Python\Python38\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


# Webcam Set Up

In [15]:
import time

# Store last seen face positions and timestamp
last_faces = []
last_logged_time = {}

# Start webcam
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Detect faces
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5)

    face_imgs = []
    face_coords = []

    for (x, y, w, h) in faces:
        face_id = f"{x}_{y}_{w}_{h}"
        recent = False

        for px, py, pw, ph in last_faces:
            if abs(x - px) < 20 and abs(y - py) < 20:
                recent = True
                break

        if recent:
            continue

        try:
            face = frame[y:y+h, x:x+w]
            face = cv2.resize(face, (128, 128))
            face = face.astype('float32') / 255.0
            face_imgs.append(face)
            face_coords.append((x, y, w, h))
            last_faces.append((x, y, w, h))
        except:
            continue

    if len(face_imgs) > 0:
        face_imgs = np.array(face_imgs)
        predicted_ages, predicted_genders = model.predict(face_imgs)
        predicted_gender_labels = [gender_labels[np.argmax(g)] for g in predicted_genders]

        for i, (x, y, w, h) in enumerate(face_coords):
            age = int(predicted_ages[i])
            gender = predicted_gender_labels[i]
            label = f"{gender}, Age: {age}"
            if age >= 60:
                label += " (Senior)"

            # Draw
            cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
            cv2.putText(frame, label, (x, y - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 255), 2)

            # Log once every 10 seconds
            face_key = f"{x}_{y}_{w}_{h}"
            current_time = time.time()
            if face_key not in last_logged_time or (current_time - last_logged_time[face_key]) > 10:
                last_logged_time[face_key] = current_time
                time_of_visit = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

                # Save to CSV
                new_entry = pd.DataFrame([[age, gender, time_of_visit]], columns=['Age', 'Gender', 'Time'])
                new_entry.to_csv(log_file, mode='a', header=False, index=False)

                # Print the result
                print(f"Detected: {gender}, Age: {age} at {time_of_visit}" + (" (Senior)" if age >= 60 else ""))

    # Show the frame
    cv2.imshow("Senior Citizen Detection", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources
cap.release()
cv2.destroyAllWindows()



1/1 [==============================] - 0s 47ms/step
Detected: Female, Age: 37 at 2025-07-25 21:42:11
1/1 [==============================] - 0s 46ms/step
Detected: Male, Age: 24 at 2025-07-25 21:42:12
1/1 [==============================] - 0s 43ms/step
Detected: Male, Age: 21 at 2025-07-25 21:42:12
1/1 [==============================] - 0s 35ms/step
Detected: Male, Age: 28 at 2025-07-25 21:42:15
1/1 [==============================] - 0s 40ms/step
Detected: Male, Age: 35 at 2025-07-25 21:42:15
1/1 [==============================] - 0s 42ms/step
Detected: Male, Age: 31 at 2025-07-25 21:42:16
1/1 [==============================] - 0s 37ms/step
Detected: Female, Age: 31 at 2025-07-25 21:42:19
